<a href="https://colab.research.google.com/github/AniketParasher/WorldCup2023_Data_Analysis_WebScraping_Python_PowerBi/blob/main/Batting_Bowling_Data_ScraingCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Batting Data Scraping

In [50]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re

## Extracting links of all matches which is stored in column name scorecard as links

In [2]:
# sending HTTP requests and handling responses in Python
webpage = requests.get('https://www.espncricinfo.com/records/tournament/team-match-results/icc-cricket-world-cup-2023-24-15338').text

In [3]:
soup = BeautifulSoup(webpage, 'lxml')

In [4]:
links = soup.find_all('a')
link_list = []
# Extract and add the href attribute (link) from each <a> tag and storing in list named = 'link_list'
for link in links:
  href = link.get('href')
  new_herf = f"https://www.espncricinfo.com{href}"
  link_list.append(new_herf)

In [5]:
# list slice to get only links of match which contains the details of that match
score_card_link = link_list[21:116:2]
score_card_link

['https://www.espncricinfo.com/series/icc-cricket-world-cup-2023-24-1367856/india-vs-australia-final-1384439/full-scorecard',
 'https://www.espncricinfo.com/series/icc-cricket-world-cup-2023-24-1367856/australia-vs-south-africa-2nd-semi-final-1384438/full-scorecard',
 'https://www.espncricinfo.com/series/icc-cricket-world-cup-2023-24-1367856/india-vs-new-zealand-1st-semi-final-1384437/full-scorecard',
 'https://www.espncricinfo.com/series/icc-cricket-world-cup-2023-24-1367856/india-vs-netherlands-45th-match-1384436/full-scorecard',
 'https://www.espncricinfo.com/series/icc-cricket-world-cup-2023-24-1367856/england-vs-pakistan-44th-match-1384435/full-scorecard',
 'https://www.espncricinfo.com/series/icc-cricket-world-cup-2023-24-1367856/australia-vs-bangladesh-43rd-match-1384434/full-scorecard',
 'https://www.espncricinfo.com/series/icc-cricket-world-cup-2023-24-1367856/afghanistan-vs-south-africa-42nd-match-1384433/full-scorecard',
 'https://www.espncricinfo.com/series/icc-cricket-worl

In [49]:
df_list = []
# loop is used to run through each match link and scarp the data of batting from that webpage
for i in score_card_link:
  webpage = requests.get(i).text
  soup = BeautifulSoup(webpage, 'lxml')
  column = []
  heading = soup.find_all('th', class_='ds-w-0')[0:7]
  for i in heading:
    column.append(i.text.strip())

  column.insert(0, 'match')
  column.insert(1, 'teaminnings')
  column.insert(2, 'battingPos')
  column.insert(4, 'dismissal')
  bat_df = pd.DataFrame(columns = column)
  team_name = soup.find_all('span', class_='ds-text-tight-l ds-font-bold ds-text-typo hover:ds-text-typo-primary ds-block ds-truncate')
  team_names_text = [team.get_text(strip=True) for team in team_name]
  match_team = ' Vs '.join(team_names_text)

  # there are divs, each div tag has one team and two table which contain batting data in one table and bowling in other table of that team
  for i in range(0,2):
    # this will target the div which are two in number
    div_2 = soup.find_all('div', class_='ds-rounded-lg ds-mt-2')[i]

    # this will store team name in the div
    team_innings1 = div_2.find('span', class_='ds-text-title-xs ds-font-bold ds-capitalize').text

    # batting data is in 1st table with index 0, index 1 has bowling data
    table_tags = div_2.find_all('table', class_='ds-w-full')[0]

    # this will collect all data from table as list
    batting_player = table_tags.find_all('td', class_='ds-min-w-max')
    # batting_player
    batting_det = []
    for i in batting_player:
      batting_det.append(i.text.strip())

    batting_details = batting_det[:-6]

    # dividing the list by converting to array to collect row wise data with each row 7 data
    parts = len(batting_details)/8 # 8 is used because we have 8 columns in the table
    arr = np.array(batting_details)
    result = np.array_split(arr, parts)
    result

    # loop on each array and converting it into list and adding in dataframe as a row
    iterator = 1
    for i in result:
      individual_list = i.tolist()
      individual_list.insert(0, match_team)
      individual_list.insert(1, team_innings1)
      individual_list.insert(2, iterator)
      length = len(bat_df)
      bat_df.loc[length] = individual_list
      iterator += 1

  df_list.append(bat_df)

In [51]:
result = pd.concat(df_list, axis=0)

In [52]:
result.head(30)

,match,teaminnings,battingPos,BATTING,dismissal,R,B,M,4s,6s,SR
0,India Vs Australia,India,1,Rohit Sharma (c),c Head b Maxwell,47,31,44,4,3,151.61
1,India Vs Australia,India,2,Shubman Gill,c Zampa b Starc,4,7,21,0,0,57.14
2,India Vs Australia,India,3,Virat Kohli,b Cummins,54,63,99,4,0,85.71
3,India Vs Australia,India,4,Shreyas Iyer,c †Inglis b Cummins,4,3,3,1,0,133.33
4,India Vs Australia,India,5,KL Rahul †,c †Inglis b Starc,66,107,133,1,0,61.68
5,India Vs Australia,India,6,Ravindra Jadeja,c †Inglis b Hazlewood,9,22,33,0,0,40.90
6,India Vs Australia,India,7,Suryakumar Yadav,c †Inglis b Hazlewood,18,28,57,1,0,64.28
7,India Vs Australia,India,8,Mohammed Shami,c †Inglis b Starc,6,10,9,1,0,60.00
8,India Vs Australia,India,9,Jasprit Bumrah,lbw b Zampa,1,3,5,0,0,33.33
9,India Vs Australia,India,10,Kuldeep Yadav,run out (Labuschagne/Cummins),10,18,28,0,0,55.55


## This data will be cleaned using python after scraping all the data.

In [46]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 876 entries, 0 to 13
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   match        876 non-null    object
 1   teaminnings  876 non-null    object
 2   battingPos   876 non-null    int64 
 3   BATTING      876 non-null    object
 4   dismissal    876 non-null    object
 5   R            876 non-null    object
 6   B            876 non-null    object
 7   M            876 non-null    object
 8   4s           876 non-null    object
 9   6s           876 non-null    object
 10  SR           876 non-null    object
dtypes: int64(1), object(10)
memory usage: 82.1+ KB


In [48]:
result.to_csv('batting_data_scraped_CSV.csv', index=False)

## 2. Bowling Data Scraping

In [ ]:
import pandas as pd
import numpy as np
import requests
import re
from bs4 import BeautifulSoup

# Cheching code, scraping data from one match page, India Vs Australia

## Original page link below:

In [ ]:
https://www.espncricinfo.com/series/icc-cricket-world-cup-2023-24-1367856/india-vs-australia-final-1384439/full-scorecard

In [ ]:
webpage = requests.get('https://www.espncricinfo.com/series/icc-cricket-world-cup-2023-24-1367856/india-vs-australia-final-1384439/full-scorecard').text
soup = BeautifulSoup(webpage, 'lxml')
column = []
heading = soup.find_all('th', class_='ds-w-0')[7:17]
for i in heading:
  column.append(i.text.strip())

column.insert(0, 'match')
column.insert(1, 'bowlingTeam')
column.insert(2, 'bowlerName')

bowl_df = pd.DataFrame(columns = column)
team_name = soup.find_all('span', class_='ds-text-tight-l ds-font-bold ds-text-typo hover:ds-text-typo-primary ds-block ds-truncate')
team_names_text = [team.get_text(strip=True) for team in team_name]
bowling_team = team_names_text[::-1]
match_team = ' Vs '.join(team_names_text)

for i in range(0,2):
  # this will target the div which are two in number
  div_23 = soup.find_all('div', class_='ds-rounded-lg ds-mt-2')[i]

  # this will focus on finding the table team name
  team_innings1 = div_23.find('span', class_='ds-text-title-xs ds-font-bold ds-capitalize').text

  table_tags = div_23.find_all('table', class_='ds-w-full')[1]
  class_pattern = re.compile(r'^ds-')
  bowling_player = table_tags.find_all('td', class_=class_pattern)

  bowling_details = []
  for j in bowling_player:
    bowling_details.append(j.text.strip())

  bowling_details
  parts = len(bowling_details)/11
  arr = np.array(bowling_details)
  result = np.array_split(arr, parts)
  result

  for j in result:
    individual_list = j.tolist()
    individual_list.insert(0, match_team)
    individual_list.insert(1, bowling_team[i])
    length = len(bowl_df)
    bowl_df.loc[length] = individual_list

In [ ]:
bowl_df

,match,bowlingTeam,bowlerName,O,M,R,W,ECON,0s,4s,6s,WD,NB
0,India Vs Australia,Australia,Mitchell Starc,10,0,55,3,5.50,30,4,1,3,0
1,India Vs Australia,Australia,Josh Hazlewood,10,0,60,2,6.00,22,4,1,1,0
2,India Vs Australia,Australia,Glenn Maxwell,6,0,35,1,5.83,19,4,1,0,0
3,India Vs Australia,Australia,Pat Cummins,10,0,34,2,3.40,30,0,0,2,0
4,India Vs Australia,Australia,Adam Zampa,10,0,44,1,4.40,22,1,0,1,0
5,India Vs Australia,Australia,Mitchell Marsh,2,0,5,0,2.50,7,0,0,0,0
6,India Vs Australia,Australia,Travis Head,2,0,4,0,2.00,8,0,0,0,0
7,India Vs Australia,India,Jasprit Bumrah,9,2,43,2,4.77,37,8,0,0,0
8,India Vs Australia,India,Mohammed Shami,7,1,47,1,6.71,28,6,1,3,0
9,India Vs Australia,India,Ravindra Jadeja,10,0,43,0,4.30,29,1,1,1,0


## Now scraping data from all 48 matches in the world cup to get the bowling data

In [ ]:
df_list2 = []

# score_card_link contains the list of links to all 48 matches
for i in score_card_link:
  webpage = requests.get(i).text
  soup = BeautifulSoup(webpage, 'lxml')
  column = []
  heading = soup.find_all('th', class_='ds-w-0')[7:17]
  for j in heading:
    column.append(j.text.strip())

  column.insert(0, 'match')
  column.insert(1, 'bowlingTeam')
  column.insert(2, 'bowlerName')

  bowl_df = pd.DataFrame(columns = column)
  team_name = soup.find_all('span', class_='ds-text-tight-l ds-font-bold ds-text-typo hover:ds-text-typo-primary ds-block ds-truncate')
  team_names_text = [team.get_text(strip=True) for team in team_name]
  bowling_team = team_names_text[::-1]
  match_team = ' Vs '.join(team_names_text)

  for k in range(0,2):
    # this will target the div which are two in number
    div_23 = soup.find_all('div', class_='ds-rounded-lg ds-mt-2')[k]

    # this will focus on finding the table team name
    team_innings1 = div_23.find('span', class_='ds-text-title-xs ds-font-bold ds-capitalize').text

    table_tags = div_23.find_all('table', class_='ds-w-full')[1]
    class_pattern = re.compile(r'^ds-')
    bowling_player = table_tags.find_all('td', class_=class_pattern)

    bowling_details = []
    for l in bowling_player:
      bowling_details.append(l.text.strip())

    bowling_details
    parts = len(bowling_details)/11
    arr = np.array(bowling_details)
    result = np.array_split(arr, parts)
    result

    for m in result:
      individual_list = m.tolist()
      individual_list.insert(0, match_team)
      individual_list.insert(1, bowling_team[k])
      length = len(bowl_df)
      bowl_df.loc[length] = individual_list

  df_list2.append(bowl_df)

In [ ]:
result1 = pd.concat(df_list2, axis=0)

In [ ]:
# result1.to_csv('bowling_data.csv', index=False)
result1

,match,bowlingTeam,bowlerName,O,M,R,W,ECON,0s,4s,6s,WD,NB
0,India Vs Australia,Australia,Mitchell Starc,10,0,55,3,5.50,30,4,1,3,0
1,India Vs Australia,Australia,Josh Hazlewood,10,0,60,2,6.00,22,4,1,1,0
2,India Vs Australia,Australia,Glenn Maxwell,6,0,35,1,5.83,19,4,1,0,0
3,India Vs Australia,Australia,Pat Cummins,10,0,34,2,3.40,30,0,0,2,0
4,India Vs Australia,Australia,Adam Zampa,10,0,44,1,4.40,22,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7,England Vs New Zealand,England,Sam Curran,6,2,47,1,7.83,17,6,1,2,0
8,England Vs New Zealand,England,Mark Wood,5,0,55,0,11.00,9,7,2,1,0
9,England Vs New Zealand,England,Moeen Ali,9.2,0,60,0,6.42,20,2,3,0,0
10,England Vs New Zealand,England,Adil Rashid,7,0,47,0,6.71,15,4,1,0,0


## Original page link of one match from where data of bowling and batting are scarped

In [ ]:
https://www.espncricinfo.com/series/icc-cricket-world-cup-2023-24-1367856/india-vs-australia-final-1384439/full-scorecard